In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import spacy
import re

In [2]:
# user warningi od spacy zaśmiecają
import warnings
warnings.filterwarnings("ignore")

In [3]:
def map_to_category(similarity: float):
    if similarity == 1.0:
        return '100%'
    elif 0.9 <= similarity < 1.0:
        return r'90% - 99%'
    elif 0.8 <= similarity < 0.9:
        return r'80% - 89%'
    else:
        return '< 80%'

### Następnym krokiem będzie polepszenie niektórych wierszy

In [4]:
df = pd.read_csv('mergeAfterAnchoring.csv', index_col=0) # z tym zakończyliśmy etap 2.
compared_df = pd.read_csv('who_said_splitted.csv', index_col=0) # ponownie do tego samego .csv będziemy porównywać
df

,start,end,who_said,line,similar_line,similarity,similarity_category
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%
...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",0.46,< 80%


In [5]:
df.similarity.describe()

count    1893.000000
mean        0.908489
std         0.142667
min         0.070000
25%         0.850000
50%         0.980000
75%         1.000000
max         1.000000
Name: similarity, dtype: float64

In [6]:
df['previously_similar'] = df['similar_line']
df

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%,Report to your stations immediately
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%,This is not a drill
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%,We are under attack
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%,Shit
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%,Language
...,...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%,They're not a team
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%,Let's beat 'em into shape
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%,Avengers
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",0.46,< 80%,"Fine, I'll do it myself"


In [7]:
'''
Szukamy wszystkich kotwic. Pomiędzy nimi będziemy szukać braków. 
Wstępnie ustalamy kotwice na wiersze, w których similiarity > 0.95

'''

df['is_anchor'] = df.similarity >= 1.0
df

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar,is_anchor
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%,Report to your stations immediately,True
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%,This is not a drill,False
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%,We are under attack,True
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%,Shit,True
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%,Language,True
...,...,...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%,They're not a team,False
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%,Let's beat 'em into shape,False
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%,Avengers,True
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",0.46,< 80%,"Fine, I'll do it myself",False


In [8]:
'''
Wybieramy wiersze, dla których szukamy lepszego dopasowania
Wstępnie jest to similarity <= 0.75

'''


df['to_replace'] = df.similarity < 0.8
df

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar,is_anchor,to_replace
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%,Report to your stations immediately,True,False
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%,This is not a drill,False,False
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%,We are under attack,True,False
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%,Shit,True,False
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%,Language,True,False
...,...,...,...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%,They're not a team,False,False
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%,Let's beat 'em into shape,False,False
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%,Avengers,True,False
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",0.46,< 80%,"Fine, I'll do it myself",False,True


In [9]:
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

def remove_punctuation(line: str) -> str:
    if isinstance(line, str):
        doc = nlp(line)
        sentence_tokens = []
        for token in doc:
            if token.text.isalnum():
                sentence_tokens.append(token.lemma_.lower())

        joined_tokens = ' '.join(sentence_tokens)
        return joined_tokens.strip()
    else:
        return ''


# aplikujemy metodę dla całej kolumny "line" jednocześnie i zapisujemy rezultat w nowej kolumnie "with_no_punctuation"
df['lemmas'] = df['line'].apply(remove_punctuation)
compared_df['lemmas'] = compared_df['line'].apply(remove_punctuation)

In [10]:
df

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar,is_anchor,to_replace,lemmas
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%,Report to your stations immediately,True,False,report to your station immediately
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%,This is not a drill,False,False,this be not a drill we be under attack
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%,We are under attack,True,False,we be under attack
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%,Shit,True,False,shit
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%,Language,True,False,language
...,...,...,...,...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%,They're not a team,False,False,they good they not a team
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%,Let's beat 'em into shape,False,False,let beat they into shape
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%,Avengers,True,False,avenger
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",0.46,< 80%,"Fine, I'll do it myself",False,True,fine


In [11]:
def count_words(line: str) -> int:
    num = len(line.split())
    return num


def shorten_text(text: str, number: int) -> str:
    text = str(text)
    splitted = text.split()
    n_splits = len(splitted)

    if number > n_splits:
        number = n_splits

    output = ' '.join(splitted[:number])
    return output


df['word_count'] = df['lemmas'].apply(count_words)

In [12]:
anchor_indexes = df[df.is_anchor].index
anchor_indexes = anchor_indexes.to_numpy()

#pd.set_option('display.max_rows', None)

def find_top_anchor(idx: int) -> int:
    if idx < 40:
        return 0
    else:
        return idx-50


def find_bot_anchor(idx: int) -> int:
    if idx > 1853:
        return 1892
    else:
        return idx+50
    

for i, row in df.iterrows():
    if row['to_replace']:
        print('='*80)
        print('Looking for: ', row['lemmas'])
        time_line = nlp(row['lemmas'])
        # w tych ramach będziemy szukać
        top_anchor = find_top_anchor(i)
        bottom_anchor = find_bot_anchor(i)
        
        subset = compared_df.iloc[top_anchor:bottom_anchor, :].copy()
        word_count = int(row['word_count'])
        print(word_count)
        subset['shortened_text'] = subset.apply(lambda x: shorten_text(x['lemmas'], word_count), axis=1)

        highest_similarity = 0
        for j, r in subset.iterrows():
            compared_text = r['shortened_text']
            compared_line = nlp(compared_text)
            similarity = time_line.similarity(compared_line)
           # print(row['lemmas'], ' = ', compared_text)
            if similarity >= highest_similarity:
                print('NEW CANDIDATE: {},\t WITH SIMILARITY: {}'.format(compared_text, similarity))
                df.at[i, 'was_replaced'] = True

                df.at[i, 'who_said'] = r['speaker']
                df.at[i, 'similarity'] = np.round(similarity, 2)
                df.at[i, 'similarity_category'] = map_to_category(np.round(similarity, 2))
                df.at[i, 'similar_line'] = r['line']
                highest_similarity = similarity
            else:
                print('WORSE: {},\t WITH SIMILARITY: {}'.format(compared_text, similarity))

        del subset
    else:
        continue

Looking for:  costel
1
Looking for:  avenger go home
3
Looking for:  you have to be fast than
6
NEW CANDIDATE: do you see a power source,	 WITH SIMILARITY: 0.4747809653598078
WORSE: there a particle wave below the,	 WITH SIMILARITY: 0.060437969387872804
WORSE: great i wanna poke it with,	 WITH SIMILARITY: 0.10123914658033285
WORSE: drawbridge be down people,	 WITH SIMILARITY: 0.34126256214579487
WORSE: the enhance,	 WITH SIMILARITY: 0.06222723435729346
WORSE: he a blur,	 WITH SIMILARITY: 0.015805302036729457
WORSE: all the new player we face,	 WITH SIMILARITY: 0.07228082479241968
WORSE: in fact i still have,	 WITH SIMILARITY: 0.24210762177576062
WORSE: clint hit pretty bad guy,	 WITH SIMILARITY: 0.06370619843922354
NEW CANDIDATE: we go to need evac,	 WITH SIMILARITY: 0.6312619239728717
WORSE: i can get barton to the,	 WITH SIMILARITY: 0.5506679269539797
WORSE: the soon we go the well,	 WITH SIMILARITY: 0.20845221498863753
WORSE: you and stark secure the scepter,	 WITH SIMILARITY: 0.119

In [13]:
df

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar,is_anchor,to_replace,lemmas,word_count,was_replaced
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%,Report to your stations immediately,True,False,report to your station immediately,5,NaN
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%,This is not a drill,False,False,this be not a drill we be under attack,9,NaN
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%,We are under attack,True,False,we be under attack,4,NaN
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%,Shit,True,False,shit,1,NaN
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%,Language,True,False,language,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%,They're not a team,False,False,they good they not a team,6,NaN
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%,Let's beat 'em into shape,False,False,let beat they into shape,5,NaN
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%,Avengers,True,False,avenger,1,NaN
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",1.00,100%,"Fine, I'll do it myself",False,True,fine,1,True


In [14]:
def mapping(x):
    if isinstance(x, bool):
        return True
    else:
        return False
df['was_replaced'] = df['was_replaced'].apply(mapping)

df

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar,is_anchor,to_replace,lemmas,word_count,was_replaced
0,"00:00:42,800","00:00:44,802",Announcer,Report to your stations immediately.,Report to your stations immediately,1.00,100%,Report to your stations immediately,True,False,report to your station immediately,5,False
1,"00:00:44,960","00:00:47,721",Announcer,This is not a drill. We are under attack!,This is not a drill,0.90,90% - 99%,This is not a drill,False,False,this be not a drill we be under attack,9,False
2,"00:00:49,120","00:00:51,088",Announcer,We are under attack!,We are under attack,1.00,100%,We are under attack,True,False,we be under attack,4,False
3,"00:01:52,200","00:01:54,487",Tony Stark,Shit!,Shit,1.00,100%,Shit,True,False,shit,1,False
4,"00:01:52,200","00:01:54,487",Steve Rogers,Language!,Language,1.00,100%,Language,True,False,language,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,"02:10:42,400","02:10:44,801",Steve Rogers,They're good. They're not a team.,They're not a team,0.97,90% - 99%,They're not a team,False,False,they good they not a team,6,False
1889,"02:10:44,960","02:10:46,610",Natasha Romanoff,Let's beat them into shape.,Let's beat 'em into shape,0.96,90% - 99%,Let's beat 'em into shape,False,False,let beat they into shape,5,False
1890,"02:11:05,080","02:11:06,605",Steve Rogers,Avengers...,Avengers,1.00,100%,Avengers,True,False,avenger,1,False
1891,"02:13:25,440","02:13:26,885",Thanos,Fine.,"Fine, I'll do it myself",1.00,100%,"Fine, I'll do it myself",False,True,fine,1,True


In [15]:
output = df[['start', 'end', 'who_said', 'line', 'similar_line', 'similarity', 'similarity_category']]
#output.to_csv('mergeAfterAnchoring.csv')

In [16]:
replaced = df[df.was_replaced==True]
replaced = replaced.loc[:, ['who_said', 'line', 'similar_line', 'previously_similar']].copy()

In [17]:
replaced

,who_said,line,similar_line,previously_similar
119,Natasha Romanoff,You'll have to be faster than...,We're gonna need evac,Well then technically you're unemployed
162,Thor,"So, yes, revels.","Yes, yes, of course","Yes, revels"
226,Dr. Helen Cho,So I've been analysing the gem inside.,"This is the next thing, Tony","Unlike you, I don't have a lot of time for par..."
227,Dr. Helen Cho,You may recognise...,Will Thor be there,Your own girlfriend won't be able to tell the ...
287,Ultron,designed by Mr Stark.,Mr,"Thanks, buddy"
...,...,...,...,...
1794,Thor,The job is finished.,The Mind Stone is the fourth of the Infinity S...,Is this the last of them
1817,Steve Rogers,Well...,"Well, we're not the '27 Yankees",Triple Yahtzee
1835,Nick Fury,"Trouble, Ms Romanoff.","Trouble, Miss Romanoff",I'm home
1877,Steve Rogers,"Family, stability...",Avengers,The simple life


In [18]:
for i, row in replaced.iterrows():
    print('''
        [{}]
        "{}"
        
        Na podstawie:
        {}
        
        Wcześniejsza opcja:
        {}  
    '''.format(row['who_said'], row['line'], row['similar_line'], row['previously_similar']))


        [Natasha Romanoff]
        "You'll have to be faster than..."
        
        Na podstawie:
        We're gonna need evac
        
        Wcześniejsza opcja:
        Well then technically you're unemployed  
    

        [Thor]
        "So, yes, revels."
        
        Na podstawie:
        Yes, yes, of course
        
        Wcześniejsza opcja:
        Yes, revels  
    

        [Dr. Helen Cho]
        "So I've been analysing the gem inside."
        
        Na podstawie:
        This is the next thing, Tony
        
        Wcześniejsza opcja:
        Unlike you, I don't have a lot of time for parties  
    

        [Dr. Helen Cho]
        "You may recognise..."
        
        Na podstawie:
        Will Thor be there
        
        Wcześniejsza opcja:
        Your own girlfriend won't be able to tell the difference  
    

        [Ultron]
        "designed by Mr Stark."
        
        Na podstawie:
        Mr
        
        Wcześniejsza opcja:
        Thank

In [19]:
replaced.shape[0]

270

In [20]:
df.similarity.describe()

count    1893.000000
mean        0.938959
std         0.097816
min         0.220000
25%         0.890000
50%         1.000000
75%         1.000000
max         1.000000
Name: similarity, dtype: float64

In [21]:
df[df.similarity<0.7]

,start,end,who_said,line,similar_line,similarity,similarity_category,previously_similar,is_anchor,to_replace,lemmas,word_count,was_replaced
45,"00:04:06,440","00:04:08,568",Iron Legion,Costel!,We wish to avoid collateral damage and will in...,0.22,< 80%,We wish to avoid collateral damage and will in...,False,True,costel,1,False
48,"00:04:11,280","00:04:13,328",Iron Legion,"Avengers, go home!",We are here to help,0.31,< 80%,We are here to help,False,True,avenger go home,3,False
119,"00:09:43,840","00:09:45,330",Natasha Romanoff,You'll have to be faster than...,We're gonna need evac,0.63,< 80%,Well then technically you're unemployed,False,True,you have to be fast than,6,True
162,"00:14:27,960","00:14:29,564",Thor,"So, yes, revels.","Yes, yes, of course",0.64,< 80%,"Yes, revels",False,True,so yes revels,3,True
226,"00:17:59,240","00:18:03,165",Dr. Helen Cho,So I've been analysing the gem inside.,"This is the next thing, Tony",0.56,< 80%,"Unlike you, I don't have a lot of time for par...",False,True,so i be analyse the gem inside,7,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,"01:57:45,320","01:57:46,481",Natasha Romanoff,Nat.,Sun's getting real low,0.69,< 80%,"I got no strings, so I have fun",False,True,nat,1,True
1740,"01:57:46,760","01:57:47,761",Clint Barton,This way.,No one eats in a dining room,0.50,< 80%,"But with Stark's stealth tech, we still can't ...",False,True,this way,2,True
1781,"02:01:53,000","02:01:54,764",Vision,Zrinka!,Humans are odd,0.66,< 80%,Right,False,True,zrinka,1,True
1782,"02:01:53,000","02:01:54,764",Zrinka,Costel! Baby!,Costel,0.66,< 80%,"But with Stark's stealth tech, we still can't ...",False,True,costel baby,2,True
